# PRE-PROCESSING THE DATA

## PREPARING THE ENVIRONMENT

Importing the libraries:

In [1]:
import pickle
import sqlite3

import pandas as pd

Defining the constants:

In [2]:
OUTPUT_PATH = '../data/estabelecimentos.db'

CHUNKSIZE = 3000000

COLS  = [0, 6, 10, 18]
NAMES = ['cnpj',
         'closing_date',
         'opening_date',
         'cep']

Creating connections and loading data:

In [3]:
conn   = sqlite3.connect('../data/estabelecimentos.db')
cursor = conn.cursor()

In [4]:
with open('../data/cnpj2id.pkl'   , 'rb') as f:
    cnpj2id = pickle.load(f)

id2cnpj = {v : k
           for k, v in cnpj2id.items()}

with open('../data/components.pkl', 'rb') as f:
    components = pickle.load(f)


cnpjs = [id2cnpj[v]
         for component in components
         for v in component]


del cnpj2id
del id2cnpj
del components

## PRE-PROCESSING THE DATA

Pre-processing the data to unify the files and filter the columns of interest:

In [5]:
for i in range(10):
    file_path = f'../data/estabelecimentos/estabelecimentos{i}.csv'

    print('Reading', file_path)

    for chunk in pd.read_csv(file_path,
                             sep=';',
                             usecols=COLS,
                             names=NAMES,
                             chunksize=CHUNKSIZE,
                             low_memory=False,
                             encoding='latin-1',
                             on_bad_lines='skip'):
        chunk = chunk[chunk.cnpj.isin(cnpjs)]
        chunk.cep = (
            chunk.cep
            .fillna('0')
            .astype(str)
            .str.replace('-', '', regex=True)
            .astype(float)
            .astype(int)
        )

        chunk.to_sql('estabelecimentos', conn, if_exists='append', index=False)

        del chunk

conn.close()

Reading ../data/estabelecimentos/estabelecimentos0.csv
Reading ../data/estabelecimentos/estabelecimentos1.csv
Reading ../data/estabelecimentos/estabelecimentos2.csv
Reading ../data/estabelecimentos/estabelecimentos3.csv
Reading ../data/estabelecimentos/estabelecimentos4.csv
Reading ../data/estabelecimentos/estabelecimentos5.csv
Reading ../data/estabelecimentos/estabelecimentos6.csv
Reading ../data/estabelecimentos/estabelecimentos7.csv
Reading ../data/estabelecimentos/estabelecimentos8.csv
Reading ../data/estabelecimentos/estabelecimentos9.csv


## EXPLORING THE DATA

Loading the data and displaying the result:

In [6]:
conn = sqlite3.connect(OUTPUT_PATH)

try:
    for i in range(10):
        file_path = f'../data/estabelecimentos/estabelecimentos{i}.csv'

        print('Reading', file_path)
        print()

        chunk = pd.read_csv(file_path,
                            sep=';',
                            usecols=COLS,
                            names=NAMES,
                            nrows=100000,
                            low_memory=False,
                            encoding='latin-1',
                            on_bad_lines='skip')
        chunk = chunk[chunk.cnpj.isin(cnpjs)]

        data_df = chunk.sample(n=1).iloc[0]
        cnpj = int(data_df.cnpj)
        cldt = int(data_df.closing_date)
        opdt = int(data_df.opening_date)
        cep  = int(data_df.cep)

        data_db = pd.read_sql_query(f"""SELECT *
                                        FROM estabelecimentos
                                        WHERE cnpj = '{cnpj}'""", conn)
        if not data_db.empty:
            data_db = data_db.iloc[0]

            cnpj_db = data_db.cnpj
            cldt_db = data_db.closing_date
            opdt_db = data_db.opening_date
            cep_db  = data_db.cep

            print(f'CNPJ         = {cnpj} ({cnpj_db})')
            print(f'Opening date = {opdt} ({opdt_db})')
            print(f'Closing date = {cldt} ({cldt_db})')
            print(f'CEP          = {cep } ({cep_db })')
        else:
            print(f'CNPJ {cnpj} not found in the database.')

        print()
        print()

        del chunk
finally:
    conn.close()

    print()
    print('Database connection closed.')

Reading ../data/estabelecimentos/estabelecimentos0.csv

CNPJ         = 38086462 (38086462)
Opening date = 20200813 (20200813)
Closing date = 20210816 (20210816)
CEP          = 15530000 (15530000)


Reading ../data/estabelecimentos/estabelecimentos1.csv

CNPJ         = 5060649 (5060649)
Opening date = 20020425 (20020425)
Closing date = 20210217 (20210217)
CEP          = 6711500 (6711500)


Reading ../data/estabelecimentos/estabelecimentos2.csv

CNPJ         = 41995139 (41995139)
Opening date = 20210518 (20210518)
Closing date = 20210518 (20210518)
CEP          = 3372000 (3372000)


Reading ../data/estabelecimentos/estabelecimentos3.csv

CNPJ         = 14522394 (14522394)
Opening date = 20111025 (20111025)
Closing date = 20240813 (20240813)
CEP          = 21340360 (21340360)


Reading ../data/estabelecimentos/estabelecimentos4.csv

CNPJ         = 53666673 (53666673)
Opening date = 19840728 (19840728)
Closing date = 20051103 (20051103)
CEP          = 4062003 (4062003)


Reading ../data/es